In [66]:
import json
from numpy import mat
import pandas as pd
from zipfile import ZipFile
import os
import requests
import kaggle

In [67]:
match_list = []
byb_list = []
match_list_22 = []
byb_list_22 = []

In [68]:
def get_match_info(data,file_name):
    try:
        match_info = data['info']
        match_info_dict =  {}
        match_info_dict['ID'] = int(file_name)
        match_info_dict['City'] = match_info['city'] if "city" in match_info else 'NA'
        match_info_dict['Date'] = match_info['dates'][0]
        match_info_dict['Season'] = match_info['season']
        match_info_dict['MatchNumber'] = match_info['event']['match_number'] if "match_number" in match_info["event"] else match_info['event']['stage']
        match_info_dict['Team1'] = match_info['teams'][0]
        match_info_dict['Team2'] = match_info['teams'][1]
        match_info_dict['Venue'] = match_info['venue']
        match_info_dict['TossWinner'] = match_info['toss']['winner']
        match_info_dict['TossDecision'] = match_info['toss']['decision']
        #TODO
        outcome = match_info['outcome']
        if ("result" in outcome):
            # A superover
            if outcome['result'] == 'tie':
                match_info_dict['SuperOver'] = 'Y'
                match_info_dict['WinningTeam'] = outcome['eliminator']
                match_info_dict['WonBy'] = 'SuperOver'
                match_info_dict['Margin'] = 'NA'
            elif outcome['result'] == 'no result':
                match_info_dict['SuperOver'] = 'NA'
                match_info_dict['WinningTeam'] = 'NA'
                match_info_dict['WonBy'] = 'NoResults'
                match_info_dict['Margin'] = 'NA'
            match_info_dict['method'] = 'NA'

        else:
            match_info_dict['SuperOver'] = 'N'
            match_info_dict['WinningTeam'] = outcome['winner']
            if "wickets" in outcome['by']:
                match_info_dict['WonBy'] = 'Wickets'
                match_info_dict['Margin'] = outcome['by']['wickets']
            else:
                match_info_dict['WonBy'] = 'Runs'
                match_info_dict['Margin'] = outcome['by']['runs']
            if "method" in outcome:
                match_info_dict['method'] = outcome['method']
            else:
                match_info_dict['method'] = 'NA'

        match_info_dict['Player_of_Match'] = match_info['player_of_match'][0] if "player_of_match" in match_info else 'NA'
        match_info_dict['Team1Players'] = match_info['players'][match_info_dict['Team1']]
        match_info_dict['Team2Players'] = match_info['players'][match_info_dict['Team2']]
        match_info_dict['Umpire1'] = match_info['officials']['umpires'][0]
        match_info_dict['Umpire2'] = match_info['officials']['umpires'][1]
        if match_info_dict['Season'] == '2022':
            match_list_22.append(match_info_dict)
        else:
            match_list.append(match_info_dict)
    except Exception as e:
        raise Exception(e)


In [69]:
def get_ball_by_ball_info(data,file_name):
    try:
        innings = data['innings']
        for i in range(len(innings)):
            overs = innings[i]['overs']
            for over in overs:
                deliveries = over['deliveries']
                for d in range(len(deliveries)):
                    byb = {}
                    byb['ID'] = int(file_name)
                    byb['innings'] = i+1
                    byb['overs'] = over['over']
                    byb['ballnumber'] = d+1
                    byb['batter'] = deliveries[d]['batter']
                    byb['bowler'] = deliveries[d]['bowler']
                    byb['non-striker'] = deliveries[d]['non_striker']
                    if "extras" in deliveries[d]:
                        byb['extra_type'] = list(deliveries[d]['extras'].keys())[0]
                    else:
                        byb['extra_type'] = 'NA'
                    byb['batsman_run'] = deliveries[d]['runs']['batter']
                    byb['extras_run'] = deliveries[d]['runs']['extras']
                    byb['total_run'] = deliveries[d]['runs']['total']
                    byb['non_boundary'] = 1 if "non_boundary" in deliveries[d]['runs'] else 0
                    if "wickets" in deliveries[d]:
                        byb['isWicketDelivery'] = 1
                        byb['player_out'] = deliveries[d]['wickets'][0]['player_out']
                        byb['kind'] = deliveries[d]['wickets'][0]['kind']
                        byb['fielders_involved'] = deliveries[d]['wickets'][0]['fielders'][0]['name'] if 'fielders' in deliveries[d]['wickets'][0] else 'NA' 
                    else:
                        byb['isWicketDelivery'] = 0
                        byb['player_out'] = 'NA'
                        byb['kind'] = 'NA'
                        byb['fielders_involved'] = 'NA'
                    byb['BattingTeam'] = innings[i]['team']
                    if data['info']['season'] == '2022':
                        byb_list_22.append(byb)
                    else:
                        byb_list.append(byb)
    except Exception as e:
        raise Exception(e)


In [70]:
def get_data(file_path,file_name):
    file_path = file_path.removesuffix('.json')
    f = open('%s.json' % file_path)
    data = json.load(f)
    get_match_info(data,file_name.removesuffix('.json'))
    get_ball_by_ball_info(data,file_name.removesuffix('.json'))

In [71]:
print('Downloading started')
url = "https://cricsheet.org/downloads/ipl_json.zip"

# Downloading the file by sending the request to the URL
req = requests.get(url)
 
# Split URL to get the file name
filename = url.split('/')[-1]
print(filename)
# Writing the file to the local file system
with open(filename,'wb') as output_file:
    output_file.write(req.content)
print('Downloading Completed')

ipl_json.zip


In [72]:
parent_path = os.path.abspath(os.path.dirname(os.path.abspath("__file__")))


In [73]:
with ZipFile(filename,'r') as zip:
    zip.extractall('ipl_json')
path = "ipl_json"
os.chdir(path)
for file in os.listdir():
    try:
        if file.endswith(".json"):
            file_path = f"{parent_path}\{path}\{file}"
            get_data(file_path,file)
    except Exception as e:
        print(f'Error for file {file} and exception {e}')
        continue

In [74]:
os.chdir(parent_path)
print('2008-2021')
print(len(match_list))
print(len(byb_list))
print('2022')
print(len(match_list_22))
print(len(byb_list_22))


2008-2021
876
208042
2022
6
1465


In [75]:
df = pd.DataFrame(match_list)
df = df.sort_values('Date', ascending=False)
df.to_csv('IPL_Matches_2008_2021.csv',index=False)

df22 = pd.DataFrame(match_list_22)
df22 = df22.sort_values('Date', ascending=False)
df22.to_csv('ipl_2022/IPL_Matches_2022.csv',index=False)

In [76]:
ball_df = pd.DataFrame(byb_list)
ball_df = ball_df.sort_values(by=['ID','innings'], ascending=[False,True])
ball_df.to_csv('IPL_Ball_by_Ball_2008_2021.csv',index=False)

ball_df22 = pd.DataFrame(byb_list_22)
ball_df22 = ball_df22.sort_values(by=['ID','innings'], ascending=[False,True])
ball_df22.to_csv('ipl_2022/IPL_Ball_by_Ball_2022.csv',index=False)

In [77]:
!kaggle datasets metadata -p ipl_2022 vora1011/ipl-2022-match-dataset

Downloaded metadata to ipl_2022\dataset-metadata.json


In [78]:
!kaggle datasets version -m "Updated Data" -p ipl_2022

Starting upload for file IPL_Ball_by_Ball_2022.csv


  0%|          | 0.00/130k [00:00<?, ?B/s]
  6%|▌         | 8.00k/130k [00:00<00:04, 30.2kB/s]
100%|██████████| 130k/130k [00:05<00:00, 23.2kB/s] 

  0%|          | 0.00/3.18k [00:00<?, ?B/s]
100%|██████████| 3.18k/3.18k [00:00<00:00, 21.0kB/s]
100%|██████████| 3.18k/3.18k [00:07<00:00, 463B/s]  



Upload successful: IPL_Ball_by_Ball_2022.csv (130KB)
Starting upload for file IPL_Matches_2022.csv
Upload successful: IPL_Matches_2022.csv (3KB)
Dataset version is being created. Please check progress at https://www.kaggle.com/vora1011/ipl-2022-match-dataset
